# Ch.6 게이트가 추가된 RNN

- `RNN`의 단점 : 시간적으로 멀리 떨어진, long term 의존 관계를 잘 학습할 수 없음

최근에는 `게이트(gate)` 구조가 더해진 `LSTM` 또는 `GRU` 계층이 많이 쓰임

## 6.1 RNN의 문제점

- `RNN`은 장기 의존 관계를 학습하기 어려움
  - `BPTT`에서 기울기 소실 혹은 기울기 폭발이 일어나기 때문에

### 6.1.1 RNN 복습

<p align = 'left'><img src="./master/images/fig 6-1.png" height=300 /></p>  


### 6.1.2 기울기 소실 또는 기울기 폭발

<p align = 'left'><img src="./master/images/fig 6-4.png" height=300 /></p>  


### 6.1.3 기울기 소실과 기울기 폭발의 원인

### 6.1.4 기울기 폭발 대책

`기울기 클리핑(gradients clipping)`

In [1]:
import numpy as np

dW1 = np.random.rand(3, 3) * 10
dW2 = np.random.rand(3, 3) * 10
grads = [dW1, dW2]
max_norm = 5.0

def clip_grads(grads, max_norm):
    total_norm = 0
    for grad in grads:
        total_norm += np.sum(grad ** 2)
    total_norm = np.sqrt(total_norm)
    
    rate = max_norm / (total_norm + 1e-6)
    if rate < 1:
        for grad in grads:
            grad *= rate

## 6.2 기울기 소실과 LSTM

### 6.2.1 LSTM의 인터페이스

<p align = 'left'><img src="./master/images/fig 6-11.png" height=300 /></p>  


- `c` : `memory cell(기억 셀)`
  - 특징 : 데이터를 LSTM 계층 내에서만 주고받는다

### 6.2.2 LSTM 계층 조립하기

<p align = 'left'><img src="./master/images/fig 6-12.png" height=300 /></p>  


$h_t$는 $c_t$에 $tanh$로 변환한 값

### 6.2.3 output 게이트

- `output 게이트` : $tanh(c_t)$의 각 원소에 대해 '그것이 다음 시각의 은닉 상태에 얼마나 중요한가'를 조정
  - 열림 상태는 입력 $x_t$와 이전 상태 $h_{t-1}$로부터 구합니다.

<p align = 'left'><img src="./master/images/e 6-1.png" width=300 /></p>  


`output 게이트`의 열림 상태를 구하는 식  
- $\sigma$ : `Sigmoid` 함수
  - 게이트의 열림 상태는 0.0 ~ 1.0 사이의 값이므로 `Sigmoid` 함수를 사용

<p align = 'left'><img src="./master/images/fig 6-15.png" height=300 /></p>  


### 6.2.4 forget 게이트

- `forget` 게이트 : $c_{t-1}$의 기억 중에서 불필요한 기억을 잊게 해주는 게이트

<p align = 'left'><img src="./master/images/fig 6-16.png" height=300 /></p>  


<p align = 'left'><img src="./master/images/e 6-3.png" width=300 /></p>  


### 6.2.5 새로운 기억 셀

- 새로 기억해야 할 정보를 `memory cell`에 추가

<p align = 'left'><img src="./master/images/fig 6-17.png" height=300 /></p>  


### 6.2.6 input 게이트

- `input` 게이트 : 새로 추가되는 기억 셀의 각 원소가 얼마나 가치있는지 판단
- 앞에서 추가한 `새로운 기억 셀`에 곱해줌

### 6.2.7 LSTM의 기울기 흐름

<p align = 'left'><img src="./master/images/fig 6-19.png" height=300 /></p>  


- `c`의 역전파 과정에서 '$+$' 노드 와 '$\times$' 노드만을 지난다
  - $+$ 노드는 기울기에 영향을 미치지 않는다
  - $\times$ 노드의 계산은 `forget` 게이트가 제어
    - '잊어야한다'고 판단한 원소의 기울기는 작게, '잊어서는 안된다'라고 판단한 원소의 기울기는 작아지지 않게

## 6.3 LSTM 구현

`LSTM` 클래스 구현 후, $T$개의 단계를 한꺼번에 처리하는 `Time LSTM` 클래스 구현

<p align = 'left'><img src="./master/images/e 6-6.png" height=200 /></p>  


위의 계산을 한꺼번에 처리할 수 있다.

<p align = 'left'><img src="./master/images/fig 6-20.png" height=500 /></p>  


<p align = 'left'><img src="./master/images/fig 6-21.png" height=400 /></p>  


In [3]:
from common.functions import sigmoid

class LSTM:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None
        
    def forward(self, x, h_prev, c_prev):
        Wx, Wh, b = self.params
        N, H = h_prev.shape
        
        A = np.matmul(x, Wx) + np.matmul(h_prev, Wh) + b
        
        # slice
        f = A[:, :H]
        g = A[:, H:2*H]
        i = A[:, 2*H:3*H]
        o = A[:, 3*H:]
        
        f = sigmoid(f)
        g = np.tanh(g)
        i = sigmoid(i)
        o = sigmoid(o)
        
        c_next = f * c_prev + g * i
        h_next = o * np.tanh(c_next)
        
        self.cache = (x, h_prev, c_prev, i, f, g, o, c_next)
        return h_next, c_next
    
    

<p align = 'left'><img src="./master/images/fig 6-22.png" height=300 /></p>  


<p align = 'left'><img src="./master/images/fig 6-23.png" height=400 /></p>  


In [4]:
dA = np.hstack((df, dg, di, do))

NameError: name 'df' is not defined

### 6.3.1 Time LSTM 구현

<p align = 'left'><img src="./master/images/fig 6-24.png" height=300 /></p>  


<p align = 'left'><img src="./master/images/fig 6-25.png" height=300 /></p>  


In [ ]:
class TimeLSTM:
    def __init__(self, Wx, Wh, b, stateful=False):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.layers = None
        self.h, self.c = None, None
        self.dh = None
        self.stateful = stateful
        
    def forward(self, xs):
        Wx, Wh, b = self.params
        N, T, D = xs.shape
        H = Wh.shape[0]
        
        self.layers = []
        hs = np.empty((N, T, D), dtype='f')
        
        if not self.stateful or self.h is None:
            self.h = np.zeros((N, H), dtype='f')
        if not self.stateful or self.c is None:
            self.c = np.zeros((N, H), dtype='f')
            
        for t in range(T):
            layer = LSTM(*self.params)
            self.h, self.c = layer.forward(xs[:, t, :], self.h, self.c)
            hs[:, t, :] = self.h
            
            self.layers.append(layer)
            
        return hs
    
    def backward(self, dhs):
        Wx, Wh, b = self.params
        N, T, H = dhs.shape
        D = Wx.shape[0]
        
        dxs = np.empty((N, T, D), dtype='f')
        dh, dc = 0, 0
        
        grads = [0, 0, 0]
        for t in reversed(range(T)):
            layer = self.